<a href="https://colab.research.google.com/github/QOneK/Ryerson-Data-Analytics-Final-Project-for-Kyuhwan-Kim/blob/master/Data_Analytics_Final_Project_for_Kyuhwan_Kim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Data Analytics Final Project for Kyuhwan Kim**

---
Dateset can be found at:
https://www.kaggle.com/aaron7sun/stocknews


# Importing files into Google Colabs

---



This code allows you to connect Google Colabs with Google Drive. This method was done so that the files do not need to be inputted everytime Colabs was run. 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


** Use this portion of code if you cannot connect Google Drive and want to manually upload the dataset files**

To use this portion of code, uncomment the # signs

In [5]:
#from google.colab import files
#uploaded = files.upload()

# Loading Python Libraries and CSV files, then running EDA (Exploratory Data Analysis)

---



Import the necessary libraries: Pandas, Numpy, Matplotlib

In [6]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification,confusion_matrix,accuracy_score

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.classification module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


Read the .csv files and put them into variables

In [8]:
reddit = pd.read_csv('/content/drive/My Drive/Coding/GitHub for Kyuhwan Kim/Ryerson Data Analytics Final Project for Kyuhwan Kim/Dataset/RedditNews.csv')
djia = pd.read_csv('/content/drive/My Drive/Coding/GitHub for Kyuhwan Kim/Ryerson Data Analytics Final Project for Kyuhwan Kim/Dataset/upload_DJIA_table.csv')
reddit_djia = pd.read_csv('/content/drive/My Drive/Coding/GitHub for Kyuhwan Kim/Ryerson Data Analytics Final Project for Kyuhwan Kim/Dataset/Combined_News_DJIA.csv')

## EDA (Exploratory Data Analysis)

### EDA for Reddit Dataframe

.info() method allowed to see what the name of attributes, number of rows and the data types of the columns



In [ ]:
reddit.info()

Used the .min() and .max() method to find the range of the dates

In [ ]:
#to find the minimum date of reddit
reddit['Date'].min()

In [ ]:
#to find the maximum date of reddit
reddit['Date'].max()

.nunique() method allowed to see how many unique dates and articles that were present

In [ ]:
#number of unique inputs 
reddit.nunique()

### EDA for DJIA Dataframe

.info() method allowed to see what the name of attributes, number of rows and the data types of the columns

In [ ]:
djia.info()

Used .min() and .max methods to find the range of the dates. 

In [ ]:
djia['Date'].min()

In [ ]:
djia['Date'].max()

.describe() method allowed to display rudimentary statistics about the datasets. Based on experimentation, this works only with numeric datatypes. 

In [ ]:
djia.describe()

Visual Representation DIJA (Dow Jones Industrial Average) "Adj Close" Trends
(It seems that there are up and down movements but a general downward trend with recovery at the end)

In [ ]:
#this takes some time to run
plt.plot(djia['Date'],djia['Adj Close'])
plt.show()

### EDA for reddit_djia Dataframe 
This is the "cleaned" dataset; combination of both news and DJIA (adj price) label. 

In [ ]:
reddit_djia.info()

In [ ]:
reddit_djia.describe()

How many of the rows are 1? 0? in the class label

In [ ]:
reddit_djia['Label'].value_counts()

There appears to be more 1 (up) than 0 (down). However, from the trendline from DJIA Adj Price, we can see that there is actually a downward trend. Perhaps the magnitude of the up/down wasn't taken into consideration. 

This is to check the date range. From observation, we can see that the date range of the final "cleaned" dataset matches that of DIJA dataset. There seems to be extra erraneous data at the news dataset. 

In [ ]:
reddit_djia['Date'].min()

In [ ]:
reddit_djia['Date'].max()

Plot Pie graph of Label
https://matplotlib.org/3.1.1/gallery/pie_and_polar_charts/pie_features.html

In [ ]:
labels = 'DJIA Up/Maintain', 'DJIA Down'
sizes= [1065/1989, 924/1989]

fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()

## Checking if there are null values 

Check if there are null values that needs to be dealt with before merging the data. We can see that there were no null values. 

In [ ]:
#check null values for reddit
reddit.isnull().values.sum()

In [ ]:
#check null values for djia
djia.isnull().values.sum()

# Combining Reddit articles with DJIA datasets to create "cleaned, final" dataset



Result: 
cleaned final dataset named "**result**"

The final cleaned dataset will be created using this procedure:

a) create an extra column called 'Label' (which are the difference between proceeding day)

b) using an if-statement, categorize:
   1 (up) when DJIA increased or maintained the same
   0 (down) when DJIA decreased

c) organize the articles into the corresponding dates. This will involve grouping the articles into similar dates. 

d) merge the DJIA 'Label' column and the articles on 'Dates' column. 

## Creating DJIA 'Label'




I have noticed that the dates of DJIA and Reddit datasets are in asc/desc order. For ease of cleaning, before the label is created, instead of desc, use asc order for the djia dataset.

In [ ]:
#https://datatofish.com/sort-pandas-dataframe/
djia.sort_values(by = ['Date'], inplace=True, ascending=True)

Copy 'Adj Close' column and run .diff() method on the column to see the day by day changes in the DJIA.

In [ ]:
djia['Diff']= djia['Adj Close']

#https://stackoverflow.com/questions/39479919/how-do-i-subtract-the-previous-row-from-the-current-row-in-a-pandas-dataframe-an
djia['Diff']=djia.Diff.diff()

In [ ]:
#This is run just in case there are N/A values. These will be filled with zeros. 
djia=djia.fillna(0)

This section will take the values of 'Diff' column and using if clauses, determine the value of the class variable.

In [ ]:
#https://guillim.github.io/pandas/2018/10/22/Pandas-if-else-on-columns.html

conditions = [
    (djia['Diff'] >= 0.0),
    (djia['Diff'] < 0.0)]
choices = [int(1),int(0)]

djia['Label'] = np.select(conditions, choices, default='null')

This section is to reset the index [count row from 0 onwards]. Prior to input of this code, the index would go in desc order from index 1989. 

In [ ]:
djia.reset_index(drop=True, inplace=True)
djia.head()

The first label by default should be 0 even though Diff = 0.0
This was manually added since the previous block of code didn't account for the exception of the first label value. 

In [ ]:
djia.iloc[0,-1]=int(0)

In [ ]:
djia.head()

Prior to casting the Label column as integer, the column was an object. Changed since it is numerical value; more specifically integer. 

In [ ]:
djia['Label'] = djia['Label'].astype('int')

## Reorganizing Reddit News Dataset

Initally, the dataset had two columns: Date, News

The dataset was formated so that each row is organized in following fashion: 
Date, Top1 ... Top25 

Basically, the news articles were compiled into one single date and outputted horizontally. 

On the news dataset, this portion of code will organize the news articles into its corresponding dates using list of lists.

In [ ]:
#This is for .at function
#https://www.geeksforgeeks.org/python-pandas-dataframe-at/

#declaration of variables
previous_date = reddit.at[0, 'Date']
newslist= []
sub_newslist = []
date_list = [previous_date]

"""
This 'for loop' will iterate from the beginning to the end of the list of dataset

Each iteration of the loop, the Date column will go down. The date variable will 
be the new date value. Also, News will be stored into sub_newslist(temp list for storing articles in a single date)
At the end of the loop, the 

The 'if statement' will be activated when the value of 'Date' changes. 

When the if statement is run:
a) the date value will be inputted towards date_list list
b) values collected in sub_newslist will be inputted into newslist list
c) sub_newslist list (used as a temp collector) will be emptied 
"""

for i in range(0,len(reddit['Date'])):
  date = reddit.at[i,'Date']
  if date != previous_date:
    date_list.append(date)
    newslist.append(sub_newslist)
    sub_newslist = []
  sub_newslist.append(reddit.at[i,'News'])
  previous_date = date

#last date needs a seperate code to add articles of final date into newslist list
newslist.append(sub_newslist)

#various print statements to check that the articles are organized correctly
print(newslist[0][0])
print(newslist[-1][-1])
print(newslist[-1])
print(len(newslist))
print(len(date_list))
print(date_list[0])
print(date_list[-1])

This portion of code will take the list and output them into a correct dataframe format

---



This code will create the labels for Top 25 articles. 

In [ ]:
"""
f is f string. allows for consistency in formatting
"""
top25_labels = [f'Top{str(integer)}' for integer in list(range(1,26))]
top25_labels

This portion of code will output the joint database.

In [ ]:
#variable declaration
relevant_dates_with_news = []
cols = []

#loop will iterate from beginning to end of the dates (created from previous block of code)
#each line of output from 'relevant_dates_with_news' will output the entire corresponding row
#to a specific date 

for i in range(0,len(date_list)):
  #date_with_news is a temporary list collector, that's why inside the loop
  date_with_news = []
  date_with_news.append(date_list[i])
  date_with_news.extend(newslist[i])
  relevant_dates_with_news.append(date_with_news)

#This portion of code makes the dataframe with the header
cols.append('Date')
cols.extend(top25_labels)
df = pd.DataFrame(columns=cols)

#The loop inside loop, relevant_dates_with_news will iterate all the dates from 
#The break component ensures that each line has max of 25 articles. 
for j in range(0, len(relevant_dates_with_news)):
  for k in range(0, len(relevant_dates_with_news[j])):
    if k > 25:
      break
    df.at[j, cols[k]] = relevant_dates_with_news[j][k]

df

## Combining the cleaned DJIA and Reddit datasets

The finalized databases are merged. The 'djia' is the base database where the newly created 'df' database will merge on 'Date' column. As of result,
we expect the erraneous dates from the Reddit News will be automatically cropped
if the date range goes beyond DJIA information.

In [ ]:
result_all = pd.merge(djia, df, how='inner', on=['Date'])
result_all
result = result_all

'result_all' will have all the necessary data. This can be used to do quantitative analysis as well. 

However, for classification, we can crop for only necessary 

In [ ]:
"""
CAUTION: Run this code ONLY once. Since, the drop code if run again will remove 
more columns than necessary

If unappropriate result appears, run the previous block of code to reset 
result variable and run this portion again

The result should have Date, Label, Top1 .. Top 25 columns
"""
result.drop(result.iloc[:, 1:8], axis=1, inplace=True)
result

This portion of code tests whether or not each row has 25 articles. 
We find that there are some null values present. 

With this info, manually checked...

**Though no columns had >25 articles, it was found that there were some dates with <25 articles.**

In [ ]:
"""
This portion of code found that BEFORE EVEN CLEANING DATA that there were some dates where it did not =25 articles

Luckily, most can be neglected since when the dates are merged, most were taken out.

However, it was later found that even after merging that some dates had <25 because of null values
"""

a = list(reddit['Date'])

a,b = np.unique(a, return_counts=True)
a[b != 25]

This was a test to see where the NaN data can be found with the final dataset.

The original dataset also had the NaN at the same locations. 
Therefore, the accuracy to replicate the finished clean data was a sucess. 

The missing data can be neglegible since there aren't many NaN
and there are enough articles for even the non =25 article dates to run machine learning. 

---



This portion of code finds the NaN values on 'result' dataframe. 

Even when the dataset is fully 'cleaned', there are some issues to be dealt with.

In [ ]:
#https://dzone.com/articles/pandas-find-rows-where-columnfield-is-null
null_columns=result.columns[result.isnull().any()]
print(result[result.isnull().any(axis=1)][null_columns].head())

In [ ]:
"""
Dates where the missing data can be found. With more detail
Scroll horizontally to the end to find some NaN values. 
"""

#https://www.shanelynn.ie/select-pandas-dataframe-rows-and-columns-using-iloc-loc-and-ix/

result.iloc[[277,348,681]]

Code to output the final cleaned data to file. 

When compared to final prepared data, the result was identical to the prepared data.

In [ ]:
result.to_excel("cleaned_data.xlsx")

# Natural Language Processing

Although the dataset has been cleaned and combined, there are still more work to be done. 

We still need to pre-process the data so that the machine learning algorithm can take the input. 

## Spliting Training and Testing Sets

According to the dataset provider, there were instructions about how to split the training and test datsets. 

*"For task evaluation, please use data from 2008-08-08 to 2014-12-31 as Training Set, and Test Set is then the following two years data (from 2015-01-02 to 2016-07-01). This is roughly a 80%/20% split."*



In [ ]:
"""
Experimented with train = result['Date'] < '2015-01-01' but this gave a boolean result.
Therefore, result[true values], the values that match the conditions were ouputted with final code.
"""

train = result[result['Date'] < '2015-01-01']
test = result[result['Date'] > '2014-12-31']

## Feature Engineering

In [ ]:
"""
When looking at the list of news, we observe that some some reason, that the news articles 
start with a lower case 'b' character followed by ' or ". 
ex. b' or b"

Therefore, first step was to remove that b' or b"
"""

result.replace("b'|b\"", " ", regex = True, inplace = True)
result.head()

In [ ]:
"""
This selects just the news articles. This is where text feature engineering 
will be focused on
"""
data = train.iloc[:,2:27]

In [ ]:
"""
Removing punctuations
"""
data.replace("[^a-zA-Z]"," ", regex = True, inplace = True)

In [ ]:
"""
Converting to lower case characters
"""

for index in data:
  data[index]=data[index].str.lower()

data.head()

In [ ]:
"""
Combining each column of articles into one paragraph
"""

headline = []

for row in range(0,len(data.index)):
  headline.append(' '.join(str(x) for x in data.iloc[row,0:25]))

headline[0]

In [ ]:
"""
Implement Bag of Words
"""
countvector = CountVectorizer(ngram_range=(2,2))
traindataset = countvector.fit_transform(headline)

In [ ]:
"""
RandomForest Classifier
"""
randomclassifier = RandomForestClassifier(n_estimators = 200, criterion = 'entropy')
randomclassifier.fit(traindataset,train['Label'])

In [ ]:
"""
Predict for the Test Dataset
"""
test_transformed = []

for row in range(0,len(test.index)):
  test_transformed.append(' '.join(str(x) for x in test.iloc[row,2:27]))

test_dataset = countvector.transform(test_transformed)

predictions = randomclassifier.predict(test_dataset)

In [ ]:
matrix = confusion_matrix(test['Label'],predictions)
print (matrix)

In [ ]:
score = accuracy_score(test['Label'],predictions)
print (score)

In [ ]:
report=classification_report(test['Label'],predictions)
print(report)